# Importing libraries

- pandas  for working with tables (DataFrames)
- numpy  for generating random data efficiently

In [2]:
import pandas as pd
import numpy as np

- Generating the number of rows

In [3]:
N = 500_000  

In [4]:
# Setting a seed so that results are reproducible
np.random.seed(42)

# Creating the dataset using a pandas DataFrame

In [5]:
data = pd.DataFrame({

    # Unique ID for each transaction
    "transaction_id": np.arange(1, N + 1),

    # Generate random dates within one year
    "date": pd.to_datetime("2023-01-01") + pd.to_timedelta(
        np.random.randint(0, 365, N), unit="D"
    ),

    # Randomly assign regions
    "region": np.random.choice(
        ["Central", "Eastern", "Western", "Northern"], N
    ),

    # Random product categories
    "product_category": np.random.choice(
        ["Food", "Electronics", "Clothing", "Furniture"], N
    ),

    # Quantity of items bought (1 to 9)
    "quantity": np.random.randint(1, 10, N),

    # Unit price between 5 and 500
    "unit_price": np.round(np.random.uniform(5, 500, N), 2),

    # Payment methods
    "payment_method": np.random.choice(
        ["Cash", "Mobile Money", "Card"], N
    ),

    # Customer type
    "customer_type": np.random.choice(
        ["Retail", "Wholesale"], N
    )
})


- Creating a derived column for total transaction amount


In [6]:
!pip3 install pyarrow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3.12 -m pip install --upgrade pip


In [7]:
# total_amount = quantity * unit_price
data["total_amount"] = data["quantity"] * data["unit_price"]

# Save the dataset as a CSV file (row-based storage)
data.to_csv("transactions.csv", index=False)

# Save the dataset as a Parquet file (column-based storage)
data.to_parquet("transactions.parquet")

print("Dataset created successfully!")

Dataset created successfully!


# File size comparison

In [9]:
import os
# Get file size of CSV in megabytes
csv_size_mb = os.path.getsize("transactions.csv") / (1024 * 1024)

# Get file size of Parquet in megabytes
parquet_size_mb = os.path.getsize("transactions.parquet") / (1024 * 1024)

print(f"CSV file size: {csv_size_mb:.2f} MB")
print(f"Parquet file size: {parquet_size_mb:.2f} MB")

CSV file size: 32.86 MB
Parquet file size: 7.03 MB


- The Parquet file is smaller than the CSV file because Parquet uses column-based storage and compression. CSV stores data as plain text, which is inefficient for large datasets. Big Data systems prefer columnar formats like Parquet because they reduce disk usage and improve query performance.

# Read performance Test

In [10]:
import time

In [12]:
# Loading the entire CSV file
start_time = time.time()
df_csv = pd.read_csv("transactions.csv")
csv_load_time = time.time() - start_time
print(f"CSV load time: {csv_load_time:.2f} seconds")

CSV load time: 1.09 seconds


In [13]:
#Loading the entire Parquet file
start_time = time.time()
df_parquet = pd.read_parquet("transactions.parquet")
parquet_load_time = time.time() - start_time
print(f"Parquet load time: {parquet_load_time:.2f} seconds")

Parquet load time: 1.51 seconds


In [14]:
#Loading only selected columns from Parquet
start_time = time.time()
df_parquet_cols = pd.read_parquet(
    "transactions.parquet",
    columns=["region", "total_amount"]
)
parquet_cols_load_time = time.time() - start_time

print(f"Parquet (2 columns) load time: {parquet_cols_load_time:.2f} seconds")

Parquet (2 columns) load time: 0.09 seconds


- Loading the Parquet file is faster than loading the CSV file because Parquet is optimized for analytical workloads. When only specific columns are selected, Parquet loads even faster and uses less memory. This demonstrates why Big Data systems avoid reading unnecessary data and instead scan only required columns.

# Results

- The CSV file size is: 32.86 MB
- The Parquet file size is: 7.03 MB
- The CSV file load time is: 1.09 seconds
- The Parquet file load time: 1.51 seconds
- The Parquet (2 columns) file load time is: 0.09 seconds

The Parquet file was significantly smaller than the CSV file (7.03 MB vs 32.86 MB). This is because Parquet uses column-based storage and compression, while CSV stores data as plain text. Big Data systems prefer Parquet to reduce storage costs and improve I/O efficiency.

Although the CSV file loaded slightly faster than the Parquet file, this is expected for moderate-sized datasets due to Parquet’s decompression overhead. However, when only two columns were selected, Parquet loaded significantly faster. This demonstrates the advantage of columnar storage in analytical workloads

# Memory Usage

- The memory used before was 90MB's loading the parquet file thus moderate
- The memory used after by the RAM was 127MB's thus high

- Memory usage increased from approximately 90 MB to 127 MB when loading the full dataset, indicating higher RAM consumption during full data scans. Column selection from Parquet reduced memory usage, highlighting why Big Data systems avoid reading unnecessary columns.

# Big data problem on a small machine

- Defining the number of rows to read at a time this is aimed at reducing memory usage 

In [15]:
CHUNK_SIZE = 50_000

- Creating the total number of records

In [16]:
# Initializing a counter for total records
total_records = 0

# Reading the CSV file in chunks
for chunk in pd.read_csv("transactions.csv", chunksize=CHUNK_SIZE):
    
    # Adding the number of rows in the current chunk
    total_records += len(chunk)
print("Total number of records:", total_records)

Total number of records: 500000


- The total number of records was calculated by reading the CSV file in chunks instead of loading the entire dataset into memory. This approach prevents memory overload and is suitable for large datasets.

# Average Transaction Value per Product category

In [17]:
# Dictionaries to store cumulative sums and counts
total_amount_sum = {}
transaction_count = {}

# Process the CSV file chunk by chunk
for chunk in pd.read_csv("transactions.csv", chunksize=CHUNK_SIZE):
    
    # Group data by product category within the chunk
    grouped = chunk.groupby("product_category")["total_amount"].agg(["sum", "count"])
    
    # Accumulate results across all chunks
    for category, row in grouped.iterrows():
        total_amount_sum[category] = total_amount_sum.get(category, 0) + row["sum"]
        transaction_count[category] = transaction_count.get(category, 0) + row["count"]

# Calculate the average transaction value per category
average_transaction_value = {
    category: total_amount_sum[category] / transaction_count[category]
    for category in total_amount_sum
}

print("Average transaction value per category:")
print(average_transaction_value)

Average transaction value per category:
{'Clothing': 1259.08098429613, 'Electronics': 1262.440670266557, 'Food': 1260.870070498439, 'Furniture': 1260.529748216433}


- The average transaction value per category was computed by aggregating sums and counts incrementally across chunks. This avoids loading the entire dataset into memory while still producing accurate results

# Top 5 regions by Total Sales

In [18]:
# Dictionary for storing the total sales per region
region_total_sales = {}

# Reading the CSV file in chunks
for chunk in pd.read_csv("transactions.csv", chunksize=CHUNK_SIZE):
    
    # Calculating total sales per region in the chunk
    grouped = chunk.groupby("region")["total_amount"].sum()
    
    # Accumulating totals across all chunks
    for region, amount in grouped.items():
        region_total_sales[region] = region_total_sales.get(region, 0) + amount

# Sorting the regions by total sales (descending) and select top 5
top_5_regions = sorted(
    region_total_sales.items(),
    key=lambda x: x[1],
    reverse=True
)[:5]

print("Top 5 regions by total sales:")
print(top_5_regions)

Top 5 regions by total sales:
[('Western', 157864758.64999998), ('Northern', 157520575.82999998), ('Central', 157511204.47), ('Eastern', 157469367.06000003)]


# Why Chunking is necessary?
- Chunking is necessary because large datasets may not fit entirely into memory. By reading small portions of data at a time, memory usage is controlled and the program remains stable.

# How This Maps to Hadoop / Big Data Systems.
- This approach is similar to how Hadoop processes data by splitting files into blocks and distributing them across nodes. Each node processes a portion of the data, and the partial results are combined to produce the final output

# Python vs R reflection

# How would this task typically be handled in R?

- In R, this task is usually handled by loading the dataset into memory using data frames. Packages such as dplyr and data.table are commonly used for data manipulation. For moderately large datasets, fread() from data.table can improve performance. However, R is largely designed for in-memory analysis. When datasets grow very large, memory limitations become a challenge. Extra tools or database connections are required to scale. This makes R less suitable for Big Data without additional infrastructure

# What breaks down when datasets become very large?

- When datasets become very large, memory becomes the main limitation. Loading the entire dataset into RAM may cause the system to slow down or crash. Operations such as grouping and sorting become computationally expensive. Disk input and output also become a bottleneck. Traditional single-machine tools struggle to scale efficiently. This is why Big Data frameworks are required

# Why does Big Data move computation to the data?

- Moving large datasets across networks is slow and costly. Big Data systems instead send computation to where the data is stored. This reduces network traffic and improves performance. It also allows parallel processing across multiple machines. By processing data locally, systems achieve better scalability and fault tolerance. This design principle is central to Big Data architectures

# Storage Architecture

- If a dataset grows to 5 TB,

# Where would it be stored?
- If the dataset grew to 5 TB, it would be stored in a distributed storage system such as Hadoop Distributed File System (HDFS) or cloud object storage like Amazon S3 or Google Cloud Storage. These systems are designed to handle large volumes of data reliably

# How would it be partitioned?
- The data would be partitioned based on logical fields such as date or region. Partitioning improves query performance by allowing systems to scan only relevant subsets of data. It also enables parallel processing across multiple nodes

# Why would a single machine fail?
- A single machine would fail due to limited disk space, memory constraints, and lack of fault tolerance. Hardware failure would result in data loss or downtime. Processing such a large dataset on one machine would also be extremely slow.

# Which Big Data technologies would become necessary?
- At least three Big Data technologies would be required. HDFS would provide distributed and fault-tolerant storage. Apache Spark would enable fast, distributed data processing. Apache Hive would allow SQL-like querying on large datasets. Together, these tools support scalable and efficient Big Data analytics.